# Rossmann Store Sales Forecast

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.
This notebook mainly focuses on the Time Series Analysis (seasonal decomposition, trends, autocorrelation).
We then disscuss advantages and drawbacks of modeling with Seasonal ARIMA and Prophet.


#### Approach:
    
I first do the habitual data treatment and cleansing.
In order to understand better the patterns of the data, I will make use of libraries like matplotlib and seaborn to deep dive cases in the dataset and give better visibility on what is happening with the different types of Rossman drug stores.
This Exploratory analysis will help me move forward with the correlation analysis and feature engineering part of the project.

#### Dataset:

<li>https://www.kaggle.com/c/rossmann-store-sales</li>

# Importing Required Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")
#Data Manipulation and Treatment
import numpy as np
import pandas as pd
from datetime import datetime
#Plotting and Visualizations
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats
import itertools
#Scikit-Learn for Modeling
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
# statistics
#from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF
# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# Loading Dataset

In [ ]:
# additional store data

store = pd.read_csv("../input/store.csv")
store.describe()
store.head()

In [ ]:
# importing train data

train = pd.read_csv("../input/train.csv", parse_dates = True, low_memory = False, index_col = 'Date')
train.describe()
train.head()

#### A quick glimpse at the data:

- Sales: the turnover for any given day (target variable).
- Customers: the number of customers on a given day.
- Open: an indicator for whether the store was open: 0 = closed, 1 = open.
- Promo: indicates whether a store is running a promo on that day.
- StateHoliday: indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays.
- SchoolHoliday: indicates if the (Store, Date) was affected by the closure of public schools.
    
We are dealing with time series data so it will probably serve us to extract dates for further analysis. We also have two likely correlated vaiables in the dataset, which can be combined into a new feature.

In [ ]:
# data extraction

train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.weekofyear


In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# adding new variable

train['SalePerCustomer'] = train['Sales']/train['Customers']
train['SalePerCustomer'].describe()

On average customers spend about 9.50$ per day. Though there are days with Sales equal to zero.

##### In this section we go through the train and store data, handle missing values and create new features for further analysis.

In [ ]:
train.isnull().sum()

In [ ]:
train.fillna(0, inplace = True)

#### ECDF: empirical cumulative distribution function
To get the first impression about continious variables in the data we can plot ECDF.

In [ ]:
sns.set(style = "ticks")# to format into seaborn 
c = '#386B7F' # basic color for plots
plt.figure(figsize = (12, 6))

plt.subplot(311)
cdf = ECDF(train['Sales'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sales'); plt.ylabel('ECDF');

# plot second ECDF  
plt.subplot(312)
cdf = ECDF(train['Customers'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Customers');

# plot second ECDF  
plt.subplot(313)
cdf = ECDF(train['SalePerCustomer'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sale per Customer');

About 20% of data has zero amount of sales/customers that we need to deal with and almost 80% of time daily amount of sales was less than 1000. So what about zero sales, is it only due to the fact that the store is closed?

##### Missing values
##### Closed stores and zero sales stores

In [ ]:
# closed stores
train[(train.Open == 0) & (train.Sales == 0)]

There're 172817 closed stores in the data. It is about 10% of the total amount of observations. To avoid any biased forecasts we will drop these values.

What about opened stores with zero sales?

In [ ]:
# opened stores with zero sales
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

Interestingly enough, there are opened store with no sales on working days. There're only 54 days in the data, so we can assume that there were external factors involved, for example manifestations.

In [ ]:
#print("Closed stores and days which didn't have any sales won't be counted into the forecasts.")

train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

print("In total: ", train.shape)

In [ ]:
train=train.drop(columns=train[(train.Open == 1) & (train.Sales == 0)].index)

In [ ]:
{"Mean":np.mean(train.Sales),"Median":np.median(train.Sales)}

In [ ]:
train.Customers.describe()

In [ ]:
{"Mean":np.mean(train.Customers),"Median":np.median(train.Customers)}

### store.csv file EDA

In [ ]:
store.head()

1. Store: a unique Id for each store
2. StoreType: differentiates between 4 different store models: a, b, c, d
3. Assortment: describes an assortment level: a = basic, b = extra, c = extended
4. CompetitionDistance: distance in meters to the nearest competitor store
5. CompetitionOpenSince[Month/Year]: gives the approximate year and month of the time the nearest competitor was opened
6. Promo2: Promo2 is a continuing a promotion for some stores: 0 = store is not participating, 1 = store is participating
7. Promo2Since[Year/Week]: describes the year and calendar week when the store started participating in Promo2
8. PromoInterval: describes the consecutive intervals Promo2 is started, naming the months the promotion is started.
    E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [ ]:
# missing values?
store.isnull().sum()

In [ ]:
# missing values in CompetitionDistance
store[pd.isnull(store.CompetitionDistance)]


Apperently this information is simply missing from the data. No particular pattern observed. In this case, it makes a complete sense to replace NaN with the median values (which is twice less that the average).

In [ ]:
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)

Continuing further with missing data. What about Promo2SinceWeek?

In [ ]:
# no promo = no information about the promo?
_ = store[pd.isnull(store.Promo2SinceWeek)]
_[_.Promo2 != 0].shape

No, if there's no Promo2 then there's no information about it. We can replace these values by zeros. The same goes for tha variables deducted from the competition, CompetitionOpenSinceMonth and CompetitionOpenSinceYear.

In [ ]:
# replace NA's by 0
store.fillna(0, inplace = True)

In [ ]:
print("Joining train set with an additional store information.")

# by specifying inner join we make sure that only those observations 
# that are present in both train and store sets are merged together
train_store = pd.merge(train, store, how = 'inner', on = 'Store')

print("In total: ", train_store.shape)
train_store.head(1000)


In [ ]:
train_store.head(100)

#### Store types
In this section we will closely look at different levels of StoreType and how the main metric Sales is distributed among them.

In [ ]:
train_store.groupby('StoreType')['Sales'].describe()

StoreType B has the highest average of Sales among all others, however we have much less data for it. So let's print an overall sum of Sales and Customers to see which StoreType is the most selling and crowded one:

In [ ]:
train_store.groupby('StoreType')['Customers', 'Sales'].sum()

Clearly stores of type A. StoreType D goes on the second place in both Sales and Customers. What about date periods? Seaborn's facet grid is the best tool for this task:

In [ ]:
# sales trends
sns.factorplot(data = train_store, x = 'Month', y = "Sales", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               color = c)

In [ ]:
# sales trends
sns.factorplot(data = train_store, x = 'Month', y = "Customers", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', 
               color = c)

All store types follow the same trend but at different scales depending on the presence of the (first) promotion Promo and StoreType itself (case for B).

###### Already at this point, we can see that Sales escalate towards Christmas holidays. But we'll talk about seasonalities and trends later in the Time Series Analysis section.

In [ ]:
# sale per customer trends
sns.factorplot(data = train_store, x = 'Month', y = "SalePerCustomer", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               color = c)

Eventhough the plots above showed StoreType B as the most selling and performant one, in reality it is not true. The highest SalePerCustomer amount is observed at the StoreType D. 

Low SalePerCustomer amount for StoreType B describes its Buyer Cart: there are a lot of people who shop essentially for "small" things (or in a little quantity). Plus we saw that overall this StoreType generated the least amount of sales and customers over the period.

In [ ]:
# customers
sns.factorplot(data = train_store, x = 'Month', y = "Sales", 
               col = 'DayOfWeek', # per store type in cols
               palette = 'plasma', 
               hue = 'StoreType',
               row = 'StoreType', # per store type in rows
               color = c)

We see that stores of StoreType C are all closed on Sundays, whereas others are most of the time opened. Interestingly enough, stores of StoreType D are closed on Sundays only from October to December.

Bt the way what are the stores which are opened on Sundays?

In [ ]:
# stores which are opened on Sundays
train_store[(train_store.Open == 1) & (train_store.DayOfWeek == 7)]['Store'].unique()

To complete our preliminary data analysis, we can add variables describing the period of time during which competition and promotion were opened:

In [ ]:
# competition open time (in months)
train_store['CompetitionOpen'] = 12 * (train_store.Year - train_store.CompetitionOpenSinceYear) + \
        (train_store.Month - train_store.CompetitionOpenSinceMonth)
    
# Promo open time
train_store['PromoOpen'] = 12 * (train_store.Year - train_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - train_store.Promo2SinceWeek) / 4.0

# replace NA's by 0
train_store.fillna(0, inplace = True)

# average PromoOpen time and CompetitionOpen time per store type
train_store.loc[:, ['StoreType', 'Sales', 'Customers', 'PromoOpen', 'CompetitionOpen']].groupby('StoreType').mean()

The most selling and crowded StoreType A doesn't appear to be the one the most exposed to competitors. Instead it's a StoreType B, which also has the longest running period of promotion.

In [ ]:
# HeatMap
# Compute the correlation matrix 
# exclude 'Open' variable
corr_all = train_store.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_all, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask, annot = True, square = True, linewidths = 0.5, ax = ax, cmap = "BrBG", fmt='.2f')      
plt.show()

As mentioned before, we have a strong positive correlation between the amount of Sales and Customers of a store. We can also observe a positive correlation between the fact that the store had a running promotion (Promo equal to 1) and amount of Customers.

However, as soon as the store continues a consecutive promotion (Promo2 equal to 1) the number of Customers and Sales seems to stay the same or even decrease, which is described by the pale negative correlation on the heatmap. The same negative correlation is observed between the presence of the promotion in the store and the day of a week.

In [ ]:
# sale per customer trends
sns.factorplot(data = train_store, x = 'DayOfWeek', y = "Sales", 
               col = 'Promo', 
               row = 'Promo2',
               hue = 'Promo2',
               palette = 'RdPu')

There are several things here:

In case of no promotion, both Promo and Promo2 are equal to 0, Sales tend to peak on Sunday (!). Though we should note that StoreType C doesn't work on Sundays. So it is mainly data from StoreType A, B and D.
On the contrary, stores that run the promotion tend to make most of the Sales on Monday. This fact could be a good indicator for Rossmann marketing campaigns. The same trend follow the stores which have both promotion at the same time (Promo and Promo2 are equal to 1).
Promo2 alone doesn't seem to be correlated to any significant change in the Sales amount. This can be also prooved by the blue pale area on the heatmap above.

### Conclusion of EDA
- The most selling and crowded StoreType is A.
- The best "Sale per Customer" StoreType D indicates to the higher Buyer Cart. We could also assume that the stores of this types are situated in the rural areas, so that customers prefer buying more but less often.
- Low SalePerCustomer amount for StoreType B indicates to the possible fact that people shop there essentially for small things. - Which can also indicate to the label of this store type - "urban" - as it's more accessible for public, and customers don't mind shopping there from time to time during a week.
- Customers tends to buy more on Mondays when there's one promotion running (Promo) and on Sundays when there is no promotion at all (both Promo and Promo1 are equal to 0).
- Promo2 alone doesn't seem to be correlated to any significant change in the Sales amount.

# Time-Series Analysis per Store Type

ARIMA is one of the most classic time series forecasting models. During the modeling process, we mainly want to find 3 parameters. Auto-regression(AR) term, namly the lags of previous value; Integral(I) term for non-stationary differencing and Moving Average(MA) for error term.
In this section, we will analyse time series data: its trends, sesonalities and autocorrelation. Usually at the end of the analysis, we are able to develop a seasonal ARIMA (Autoregression Integrated Moving Average) model

### Seasonality

We take four stores from store types to represent their group:

- Store number 2 for StoreType A
- Store number 85 for StoreType B,
- Store number 1 for StoreType C
- Store number 13 for StoreType D.

It also makes sense to downsample the data from days to weeks using the resample method to see the present trends more clearly.

In [ ]:
train_store.head()

In [ ]:
# preparation: input should be float type
train['Sales'] = train['Sales'] * 1.0

# store types
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales']
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# store types
sales_a.resample('W').sum().plot(color = c, ax = ax1)
sales_b.resample('W').sum().plot(color = c, ax = ax2)
sales_c.resample('W').sum().plot(color = c, ax = ax3)
sales_d.resample('W').sum().plot(color = c, ax = ax4)

Retail sales for StoreType A and C tend to peak for the Christmas season and then decline after the holidays. We might have seen the same trend for StoreType D (at the bottom) but there is no information from July 2014 to January 2015 about these stores as they were closed.

### Yearly trend
The next thing to check the presence of a trend in series.
Another tool to visualize the data is the seasonal_decompose function in statsmodel. With this, the trend and seasonality become even more obvious.

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', freq = 365)
decomposition_a.trend.plot(color = c, ax = ax1)

decomposition_b = seasonal_decompose(sales_b, model = 'additive', freq = 365)
decomposition_b.trend.plot(color = c, ax = ax2)

decomposition_c = seasonal_decompose(sales_c, model = 'additive', freq = 365)
decomposition_c.trend.plot(color = c, ax = ax3)

decomposition_d = seasonal_decompose(sales_d, model = 'additive', freq = 365)
decomposition_d.trend.plot(color = c, ax = ax4)

Overall sales seems to increase, however not for the StoreType C (a third from the top). Eventhough the StoreType A is the most selling store type in the dataset, it seems that it cab follow the same decresing trajectory as StoreType C did.

## Stationarize the data:

When running a linear regression the assumption is that all of the observations are all independent of each other. In a time series, however, we know that observations are time dependent. It turns out that a lot of nice results that hold for independent random variables (law of large numbers and central limit theorem to name a couple) hold for stationary random variables. So by making the data stationary, we can actually apply regression techniques to this time dependent variable.

There are two ways you can check the stationarity of a time series. The first is by looking at the data. By visualizing the data it should be easy to identify a changing mean or variation in the data. For a more accurate assessment there is the Dickey-Fuller test.

if the ‘Test Statistic’ is greater than the ‘Critical Value’ than the time series is stationary. Below is code that will help you visualize the time series and test for stationarity.

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries, window = 12, cutoff = 0.01):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20 )
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < cutoff:
        print('p-value = %.4f. The series is likely stationary.' % pvalue)
    else:
        print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
    print(dfoutput)

In [ ]:
from scipy import stats
from scipy.stats import normaltest
def residual_plot(model):

    resid = model.resid
    print(normaltest(resid))
    # returns a 2-tuple of the chi-squared statistic, and the associated p-value. the p-value is very small, meaning
    # the residual is not a normal distribution

    fig = plt.figure(figsize=(12,8))
    ax0 = fig.add_subplot(111)

    sns.distplot(resid ,fit = stats.norm, ax = ax0) # need to import scipy.stats

    # Get the fitted parameters used by the function
    (mu, sigma) = stats.norm.fit(resid)

    #Now plot the distribution using 
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')
    plt.ylabel('Frequency')
    plt.title('Residual distribution')


    # ACF and PACF
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_acf(model.resid, lags=40, ax=ax1)
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_pacf(model.resid, lags=40, ax=ax2)

In [ ]:
test_stationarity(sales_a)

So now we need to transform the data to make it more stationary. There are various transformations you can do to stationarize the data.
The first thing we want to do is take a first difference of the data. This should help to eliminate the overall trend from the data.

In [ ]:
first_diff_a = sales_a - sales_a.shift(1)
first_diff_a = first_diff_a.dropna(inplace = False)
test_stationarity(first_diff_a, window = 12)

In [ ]:
test_stationarity(sales_b)

In [ ]:
first_diff_b = sales_b - sales_b.shift(1)
first_diff_b = first_diff_b.dropna(inplace = False)
test_stationarity(first_diff_b, window = 12)

In [ ]:
test_stationarity(sales_c)

In [ ]:
first_diff_c = sales_c - sales_c.shift(1)
first_diff_c = first_diff_c.dropna(inplace = False)
test_stationarity(first_diff_c, window = 12)

In [ ]:
test_stationarity(sales_d)

In [ ]:
first_diff_d = sales_d - sales_d.shift(1)
first_diff_d = first_diff_d.dropna(inplace = False)
test_stationarity(first_diff_d, window = 12)

#### Plot the ACF and PACF charts and find the optimal parameters

The next step is to determine the tuning parameters of the model by looking at the autocorrelation and partial autocorrelation graphs. There are many rules and best practices about how to select the appropriate AR, MA, SAR, and MAR terms for the model. The chart below provides a brief guide on how to read the autocorrelation and partial autocorrelation graphs to select the proper terms. The big issue as with all models is that you don’t want to overfit your model to the data by using too many terms.

The next step in ourtime series analysis is to review Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots.

ACF is a measure of the correlation between the timeseries with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’tn’ with series at instant ‘t1-5’…’tn-5’ (t1-5 and tn being end points).

PACF, on the other hand, measures the correlation between the timeseries with a lagged version of itself but after eliminating the variations explained by the intervening comparisons. Eg. at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

In [ ]:
# figure for subplots
plt.figure(figsize = (12, 8))

# acf and pacf for A
plt.subplot(421); plot_acf(sales_a, lags = 50, ax = plt.gca(), color = c)
plt.subplot(422); plot_pacf(sales_a, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for B
plt.subplot(423); plot_acf(sales_b, lags = 50, ax = plt.gca(), color = c)
plt.subplot(424); plot_pacf(sales_b, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for C
plt.subplot(425); plot_acf(sales_c, lags = 50, ax = plt.gca(), color = c)
plt.subplot(426); plot_pacf(sales_c, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for D
plt.subplot(427); plot_acf(sales_d, lags = 50, ax = plt.gca(), color = c)
plt.subplot(428); plot_pacf(sales_d, lags = 50, ax = plt.gca(), color = c)

plt.show()

In [ ]:
# figure for subplots
plt.figure(figsize = (12, 8))

# acf and pacf for A
plt.subplot(421); plot_acf(first_diff_a, lags = 50, ax = plt.gca(), color = c)
plt.subplot(422); plot_pacf(first_diff_a, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for B
plt.subplot(423); plot_acf(first_diff_b, lags = 50, ax = plt.gca(), color = c)
plt.subplot(424); plot_pacf(first_diff_b, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for C
plt.subplot(425); plot_acf(first_diff_c, lags = 50, ax = plt.gca(), color = c)
plt.subplot(426); plot_pacf(first_diff_c, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for D
plt.subplot(427); plot_acf(first_diff_d, lags = 50, ax = plt.gca(), color = c)
plt.subplot(428); plot_pacf(first_diff_d, lags = 50, ax = plt.gca(), color = c)

plt.show()

We can read these plots horizontally. Each horizontal pair is for one 'StoreType', from A to D. In general, those plots are showing the correlation of the series with itself, lagged by x time units correlation of the series with itself, lagged by x time units.

There is at two things common for each pair of plots: non randomnes of the time series and high lag-1 (which will probably need a higher order of differencing d/D).

- Type A and type B: Both types show seasonalities at certain lags. For type A, it is each 12th observation with positives spikes at the 12 (s) and 24(2s) lags and so on. For type B it's a weekly trend with positives spikes at the 7(s), 14(2s), 21(3s) and 28(4s) lags.
- Type C and type D: Plots of these two types are more complex. It seems like each observation is coorrelated to its adjacent observations.

### Build Model:

How to determin p, d, q
It's easy to determin I. In our case, we see the first order differencing make the ts stationary. I = 1.

AR model might be investigated first with lag length selected from the PACF or via empirical investigation. In our case, it's clearly that within 11 lags the AR is significant. Which means, we can use AR = 11

To avoid the potential for incorrectly specifying the MA order (in the case where the MA is first tried then the MA order is being set to 0), it may often make sense to extend the lag observed from the last significant term in the PACF.


In [ ]:
arima_mod_a = sm.tsa.ARIMA(sales_a, (11,1,0)).fit(disp=False)
print(arima_mod_a.summary())

What is interesting is that when the AR model is appropriately specified, the the residuals from this model can be used to directly observe the uncorrelated error. This residual can be used to further investigate alternative MA and ARMA model specifications directly by regression.

In [ ]:
residual_plot(arima_mod_a)

In [ ]:
sarima_mod_a = sm.tsa.statespace.SARIMAX(sales_a, trend='n', order=(11,1,0), seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_a.summary())

In [ ]:
residual_plot(sarima_mod_a)

In [ ]:
print(sales_a.shape)
sales_a.head()

In [ ]:
sales_a_reindex = sales_a.reindex(index=sales_a.index[::-1])

In [ ]:
sales_a_reindex

In [ ]:
mydata_a = sales_a_reindex
#mydata_a = sales_a_reindex.loc['2013-01-02':'2015-01-21']
#mydata_test = data.loc['2017-01-01':]

In [ ]:
print(mydata_a)

In [ ]:
temp_df =pd.DataFrame(mydata_a)

In [ ]:
mydata_a = temp_df

In [ ]:
sarima_mod_a_train = sm.tsa.statespace.SARIMAX(mydata_a, trend='n', order=(11,1,0), seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_a_train.summary())

In [ ]:
residual_plot(sarima_mod_a_train)

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(mydata_a, c='red')
plt.plot(sarima_mod_a_train.fittedvalues, c='blue')
plt.ylabel("Sales")
plt.xlabel("Time")

In [ ]:
#forecast = sarima_mod_a_train.predict(start =mydata_a.loc['2015-01-21':], dynamic= True)  
#plt.plot(mydata_a.loc['2013-01-02':'2015-01-21'])
plt.figure(figsize=(30,10))
forecast = sarima_mod_a_train.predict(start = 625, end = 783, dynamic= False)  
plt.plot(mydata_a.iloc[1:625])
plt.plot(forecast, c = "red")
forecast
#pred_ci = forecast.conf_int()
#pred_ci.head()

#start_index = 624
#end_index = 784
#mydata_a['forecast'] = sarima_mod_a_train.predict(start = start_index, end= end_index, dynamic= True)  
#mydata_a[start_index:end_index][['sales', 'forecast']].plot(figsize=(12, 8))

In [ ]:
arima_mod_b = sm.tsa.ARIMA(sales_b, (1,1,0)).fit(disp=False)
print(arima_mod_b.summary())

In [ ]:
residual_plot(arima_mod_b)

In [ ]:
sarima_mod_b = sm.tsa.statespace.SARIMAX(sales_b, trend='n', order=(11,1,0), seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_b.summary())

In [ ]:
residual_plot(sarima_mod_b)

In [ ]:
print(sales_b.shape)
sales_b.head()

In [ ]:
sales_b_reindex = sales_b.reindex(index=sales_b.index[::-1])

In [ ]:
#sales_b_reindex.head(100)

In [ ]:
mydata_b = sales_b_reindex

In [ ]:
temp_df =pd.DataFrame(mydata_b)

In [ ]:
mydata_b = temp_df

In [ ]:
sarima_mod_b_train = sm.tsa.statespace.SARIMAX(mydata_b, trend='n', order=(11,1,0), seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_b_train.summary())

In [ ]:
residual_plot(sarima_mod_b_train)

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(mydata_b, c='red')
plt.plot(sarima_mod_b_train.fittedvalues, c='blue')
plt.ylabel("Sales")
plt.xlabel("Time")

In [ ]:
plt.figure(figsize=(30,10))
forecast = sarima_mod_b_train.predict(start = 755, end = 941, dynamic= False)  
plt.plot(mydata_b.iloc[1:755])
plt.plot(forecast, c = "red")
forecast

In [ ]:
arima_mod_c = sm.tsa.ARIMA(sales_c, (11,1,0)).fit(disp=False)
print(arima_mod_c.summary())

In [ ]:
residual_plot(arima_mod_c)

In [ ]:
sarima_mod_c = sm.tsa.statespace.SARIMAX(sales_c, trend='n', order=(11,1,0)).fit()
print(sarima_mod_c.summary())

In [ ]:
residual_plot(sarima_mod_c)

In [ ]:
sales_c_reindex = sales_c.reindex(index=sales_c.index[::-1])

In [ ]:
mydata_c = sales_c_reindex

In [ ]:
temp_df =pd.DataFrame(mydata_c)

In [ ]:
mydata_c = temp_df

In [ ]:
sarima_mod_c_train = sm.tsa.statespace.SARIMAX(mydata_c, trend='n', order=(11,1,0), seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_c_train.summary())

In [ ]:
residual_plot(sarima_mod_c_train)

In [ ]:
print(sales_c.shape)
sales_c.head()

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(mydata_c, c='red')
plt.plot(sarima_mod_c_train.fittedvalues, c='blue')
plt.ylabel("Sales")
plt.xlabel("Time")

In [ ]:
plt.figure(figsize=(30,10))
forecast = sarima_mod_c_train.predict(start = 625, end = 780, dynamic= False)  
plt.plot(mydata_c.iloc[1:625])
plt.plot(forecast, c = "red")
forecast

In [ ]:
arima_mod_d = sm.tsa.ARIMA(sales_d, (11,1,0)).fit(disp=False)
print(arima_mod_d.summary())

In [ ]:
residual_plot(arima_mod_d)

In [ ]:
sarima_mod_d = sm.tsa.statespace.SARIMAX(sales_d, trend='n', order=(11,1,0),seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_d.summary())

In [ ]:
residual_plot(sarima_mod_d)

In [ ]:
print(sales_d.shape)
sales_d.head()

In [ ]:
sales_d_reindex = sales_d.reindex(index=sales_d.index[::-1])

In [ ]:
mydata_d = sales_d_reindex

In [ ]:
temp_df =pd.DataFrame(mydata_d)

In [ ]:
mydata_d = temp_df

In [ ]:
sarima_mod_d_train = sm.tsa.statespace.SARIMAX(mydata_d, trend='n', order=(11,1,0),seasonal_order=(2,1,0,12)).fit()
print(sarima_mod_d_train.summary())

In [ ]:
residual_plot(sarima_mod_d_train)

In [ ]:
# Stote type D
plt.figure(figsize=(50,10))
plt.plot(mydata_d, c='red')
plt.plot(sarima_mod_d_train.fittedvalues, c='blue')
plt.ylabel("Sales")
plt.xlabel("Time")

In [ ]:
plt.figure(figsize=(30,10))
forecast = sarima_mod_d_train.predict(start = 495, end = 620, dynamic= False)  
plt.plot(mydata_d.iloc[1:495])
plt.plot(forecast, c = "red")
forecast

#### Conclusion

#### Pros
- Arima can catch interactions between external features, which could improve the forecasting power of a model, But in case of Facebook prophet we cant use interactions term.

- Eventhough Prophet offers an automated solution for ARIMA, this methodology is under development and not completely stable.

#### Cons
- Fitting seasonal ARIMA model needs 4 to 5 whole seasons in the dataset, which can be the the biggest drawback for new companies.

- Seasonal ARIMA in Python has 7 hyperparameters which can be tuned only manually affecting significantly the speed of the forecasting process.

Want to see more of Kernels like this one? Leave an upvote then.